In [1]:
import csv
import json
import pandas as pd
import nltk
import numpy as np
import re
import pickle
import joblib
import matplotlib.pyplot as plt
import os
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
google_vec_file = '~/Downloads/GoogleNews-vectors-negative300.bin'
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

### Extracting & Cleaning

In [100]:
messages_whole = pd.concat([messages1['message'],messages2['message'],messages3['message']])

In [19]:
def text_df(messages):
    text = []
    button = []
    for entry in messages:
        try:
            text.append(entry['message']['text'])
        except:
            button.append(entry)
    print(len(text)/len(button))
    return pd.DataFrame(text,columns=['text'])

text = text_df(messages_whole)

0.4898778072007074

#### Observation: 1/3 of interactions were text messages.

In [4]:
def clean_messages(messages):
    messs = []
    for i in messages['message']:
        try:
            messs.append(i['message']['text'])
        except:
            pass
    messages['messages'] = messs
    
    recip = []
    for i in messages['message']:
        try:
            recip.append(i['recipient']['id'])
        except:
            pass
    messages['recipient'] = recip
    messages['recipient'] = messages['recipient'].astype('str')
    
    send = []
    for i in messages['message']:
        try:
            send.append(i['sender']['id'])
        except:
            pass
    messages['sender'] = send
    messages['sender'] = messages['sender'].astype('str')
    
    messages['rp_page'] = messages['recipient']
    messages['ru_page'] = messages['recipient']
    messages['sp_page'] = messages['sender']
    messages['su_page'] = messages['sender']
    
    messages['rp_follow'] = messages['recipient']
    messages['ru_follow'] = messages['recipient']
    messages['sp_follow'] = messages['sender']
    messages['su_follow'] = messages['sender']
    
    messages = messages.rename(index=str, columns={"date": "datemess"})
    
    return messages.drop(columns=['_id', '__v', 'message','senderId'])

messages = clean_messages(messages_whole)

In [5]:
def clean_followers(folls):
    folls = folls.drop(columns=['__v','attributes','_id','subscriptionType'])
    folls = folls.set_index(['profile'])
    folls = folls.filter(like="'gender':",axis=0)
    folls = folls.filter(like="'timezone':",axis=0)
    folls = folls.filter(like="'locale':",axis=0)
    folls = folls.reset_index()
    
    gender = []
    for i in folls['profile']:
        try:
            gender.append(i['gender'])
        except:
            pass
    folls['gender'] = gender
    
    timezone = []
    for i in folls['profile']:
        try:
            timezone.append(i['timezone'])
        except:
            pass
    folls['timezone'] = timezone
    
    locale = []
    for i in folls['profile']:
        try:
            locale.append(i['locale'])
        except:
            pass
    folls['locale'] = locale
    
    folls['pageId'] = folls['pageId'].astype('str')
    folls['userId'] = folls['userId'].astype('str')
    
    folls['rp_follow'] = folls['pageId']
    folls['ru_follow'] = folls['userId']
    folls['sp_follow'] = folls['pageId']
    folls['su_follow'] = folls['userId']
    
    return folls.drop(columns=['profile'])

followers = clean_followers(folls)

In [6]:
def clean_pages(page):
    
    page = page.drop(columns=['__v','_id','accessToken','administrators','botId','fetchedConversations','insufficientPermissions','insufficientPermissionsLastChecked','tags','whitelistedDomains'])
    
    fans = []
    for i in page['information']:
        try:
            fans.append(i['fanCount'])
        except:
            fans.append("NaN")
    page['fans'] = fans

    pagename = []
    for i in page['information']:
        try:
            pagename.append(i['username'])
        except:
            pagename.append("NaN")
    page['pagename'] = pagename
    page = page.drop(columns=['information'])
    
    for i in page['pagename']:
        if i == "NaN":
            page['pagename'] = page['name']
    page = page.drop(columns=['name'])
    
    page['pageId'] = page['pageId'].astype('str')
    page['userId'] = page['userId'].astype('str')
    
    page['rp_page'] = page['pageId']
    page['ru_page'] = page['userId']
    page['sp_page'] = page['pageId']
    page['su_page'] = page['userId']
    
    page = page.rename(index=str, columns={"date": "datepage"})
    
    return page

pages = clean_pages(page)

In [82]:
messpage = pd.merge(messages, pages, how='inner', on=['rp_page'])

df_f = pd.merge(messpage, folls, how='inner', on=['su_follow'])

df_final = df_f.drop(columns=['pageId_x','recipient','sender','ru_page_x','sp_page_x','su_page_x','rp_follow_x',
               'pageId','userId_y','rp_follow_y','ru_follow_y','sp_follow_y','ru_follow_x','sp_follow_x',
                   'pageId_y','userId_x','ru_page_y','sp_page_y','su_page_y'])

df_final.drop_duplicates(inplace=True)

#### Store data in pickle files.

In [77]:
with open('mergedmess.pkl','wb') as file:
    pickle.dump(df_final,file)
    
# with open('mergedmess.pkl','rb') as file:
#     mergedmess = pickle.load(file)